In [ ]:
! pip install transformers[sentencepiece] datasets nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from transformers import AdamWeightDecay
import tensorflow as tf

# Converting Corpus Into Sentences

In [ ]:
english_language_file = '/content/drive/MyDrive/NLP_Theory_DA/en-hi/train.en'
hindi_language_file = '/content/drive/MyDrive/NLP_Theory_DA/en-hi/train.hi'
with open(english_language_file, 'r') as file:
    english = file.readlines()
with open(hindi_language_file, 'r') as file:
    hindi = file.readlines()

In [ ]:
english_sentences_train = english[:200000]
hindi_sentences_train = hindi[:200000]
english_sentences_test = english[200001:260001]
hindi_sentences_test = hindi[200001:260001]
english_sentences_val = english[260001:320001]
hindi_sentences_val = hindi[260001:320001]
english_sentences_train = [sentence.rstrip('\n') for sentence in english_sentences_train]
hindi_sentences_train = [sentence.rstrip('\n') for sentence in hindi_sentences_train]
english_sentences_test = [sentence.rstrip('\n') for sentence in english_sentences_test]
hindi_sentences_test = [sentence.rstrip('\n') for sentence in hindi_sentences_test]
english_sentences_val = [sentence.rstrip('\n') for sentence in english_sentences_val]
hindi_sentences_val = [sentence.rstrip('\n') for sentence in hindi_sentences_val]

In [ ]:
english_sentences_train[:2]

["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.']

In [ ]:
hindi_sentences_train[:2]

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है']

In [ ]:
english_sentences_test[:2]

['After all, why not?',
 'The victim approached police and lodged complaint against the accused.']

In [ ]:
hindi_sentences_test[:2]

['आखिर हो भी न क्यों ?',
 'फरियादी ने आरोपी के खिलाफ पुलिस में शिकायत दर्ज कराई।']

In [ ]:
english_sentences_val[:2]

['Center to Top',
 'His thinking of a better future is immaculate in every field that the idea goes to the base and strengthens the roots.']

In [ ]:
hindi_sentences_val[:2]

['बीच से ऊपर',
 'बेहतर भविष्य की उनकी सोच हर क्षेत्र में बेदाग है कि यह विचार आधार पर जाता है और जड़ों को मजबूत करता है।']

# Data-Preprocessing

In [ ]:
sentence_tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

In [ ]:
train_data = {'translation': [{'en': l1, 'hi': l2} for l1, l2 in zip(english_sentences_train, hindi_sentences_train)]}
test_data = {'translation': [{'en': l1, 'hi': l2} for l1, l2 in zip(english_sentences_test, hindi_sentences_test)]}
val_data = {'translation': [{'en': l1, 'hi': l2} for l1, l2 in zip(english_sentences_val, hindi_sentences_val)]}

In [ ]:
from datasets import Dataset
train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)
val_data = Dataset.from_dict(val_data)

In [ ]:
print(train_data[:2])
print(test_data[:2])
print(val_data[:2])

{'translation': [{'en': "However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles", 'hi': 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।'}, {'en': 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.', 'hi': 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है'}]}
{'translation': [{'en': 'After all, why not?', 'hi': 'आखिर हो भी न क्यों ?'}, {'en': 'The victim approached police and lodged complaint against the accused.', 'hi': 'फरियादी ने आरोपी के खिलाफ पुलिस में शिकायत दर्ज कराई।'}]}
{'translation': [{'en': 'Center to Top', 'hi': 'बीच से ऊपर'}, {'en': 'His thinking of a better future is immaculate in every field tha

In [ ]:
def preprocessing(examples):
    inputs = [ex['en'] for ex in examples["translation"]]
    targets = [ex['hi'] for ex in examples["translation"]]
    model_inputs = sentence_tokenizer(inputs, max_length = 128, truncation = True)

    with sentence_tokenizer.as_target_tokenizer():
        labels = sentence_tokenizer(targets, max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_tokens = train_data.map(preprocessing, batched=True)
test_tokens = test_data.map(preprocessing, batched=True)
val_tokens = val_data.map(preprocessing, batched=True)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

# Model Building

In [ ]:
transformer = TFAutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
batch = 32
lr = 2e-5
wd = 0.01
epochs = 10
dc = DataCollatorForSeq2Seq(sentence_tokenizer, model = transformer, return_tensors = "tf")
gdc = DataCollatorForSeq2Seq(sentence_tokenizer, model=transformer, return_tensors = "tf", pad_to_multiple_of = 128)

In [ ]:
train_dataset = transformer.prepare_tf_dataset(
    test_tokens,
    batch_size = batch,
    shuffle = True,
    collate_fn = dc,
)

In [ ]:
validation_dataset = transformer.prepare_tf_dataset(
    val_tokens,
    batch_size = batch,
    shuffle = False,
    collate_fn = dc,
)

In [ ]:
generation_dataset = transformer.prepare_tf_dataset(
    val_tokens,
    batch_size = 8,
    shuffle = False,
    collate_fn = gdc,
)

In [ ]:
adam_optimizer = AdamWeightDecay(learning_rate = lr, weight_decay_rate = wd)
transformer.compile(optimizer = adam_optimizer)

# Model Training

In [ ]:
transformer.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/10
937/937 [==============================] - 879s 908ms/step - loss: 3.8364 - val_loss: 3.4298
Epoch 2/10
937/937 [==============================] - 837s 894ms/step - loss: 3.3787 - val_loss: 3.2610
Epoch 3/10
937/937 [==============================] - 838s 894ms/step - loss: 3.1237 - val_loss: 3.1500
Epoch 4/10
937/937 [==============================] - 826s 882ms/step - loss: 2.9252 - val_loss: 3.0822
Epoch 5/10
937/937 [==============================] - 816s 871ms/step - loss: 2.7556 - val_loss: 3.0325
Epoch 6/10
937/937 [==============================] - 819s 875ms/step - loss: 2.6133 - val_loss: 3.0029
Epoch 7/10
937/937 [==============================] - 834s 890ms/step - loss: 2.4816 - val_loss: 2.9802
Epoch 8/10
937/937 [==============================] - 838s 894ms/step - loss: 2.3618 - val_loss: 2.9601
Epoch 9/10
937/937 [==============================] - 831s 888ms/step - loss: 2.2537 - val_loss: 2.9470
Epoch 10/10
937/937 [==============================] - 839s 896m

# Results

In [ ]:
input_text  = "His name is Arun."
tokenized = sentence_tokenizer([input_text], return_tensors = 'np')
out = transformer.generate(**tokenized, max_length = 128)
res = ''
with sentence_tokenizer.as_target_tokenizer():
    res = sentence_tokenizer.decode(out[0], skip_special_tokens = True)
    print(sentence_tokenizer.decode(out[0], skip_special_tokens = True))

उसका नाम अरुण है।


# Evaluation Metrics

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
ref = ['उसका नाम अरुण है।'.split()]
candi = res.split()
print('BLEU score -> {}'.format(sentence_bleu(ref, candi)))

BLEU score -> 1.0
